In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely

In [39]:
valid_admin = pd.read_csv(f'base_info/admin2_unique.csv', header=None)[0].tolist()
    
# Read shapefile
admin_shp = gpd.read_file(f"dengue_indonesia_yifan/data/shapefiles/admin2/admin2_38.shp")
# drop invalid admin2 regions
admin_shp = admin_shp[admin_shp[f'admin2'].isin(valid_admin)]
admin_shp.sort_values('admin2', inplace=True)
admin_shp.reset_index(drop=True, inplace=True)

In [40]:
admin_shp

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,idadmin1,idadmin2,admin1,admin2,region,geometry
0,Aceh Barat,None,22746128B65593111718524,IDN,ADM2,11,1107,ACEH,ACEH BARAT,SUMATERA,"POLYGON ((96.49109 4.62277, 96.49445 4.60326, ..."
1,Aceh Barat Daya,None,22746128B227561513795,IDN,ADM2,11,1112,ACEH,ACEH BARAT DAYA,SUMATERA,"MULTIPOLYGON (((96.80559 3.71758, 96.80286 3.7..."
2,Aceh Besar,None,22746128B90547447297479,IDN,ADM2,11,1108,ACEH,ACEH BESAR,SUMATERA,"MULTIPOLYGON (((95.20544 5.28072, 95.20757 5.2..."
3,Aceh Jaya,None,22746128B21424895456858,IDN,ADM2,11,1116,ACEH,ACEH JAYA,SUMATERA,"MULTIPOLYGON (((95.58431 4.61495, 95.58182 4.6..."
4,Aceh Selatan,None,22746128B12428400263117,IDN,ADM2,11,1103,ACEH,ACEH SELATAN,SUMATERA,"MULTIPOLYGON (((97.59461 2.80777, 97.59269 2.8..."
...,...,...,...,...,...,...,...,...,...,...,...
509,Way Kanan,None,22746128B8708218473228,IDN,ADM2,18,1807,LAMPUNG,WAY KANAN,SUMATERA,"POLYGON ((104.50089 -4.95499, 104.50019 -4.949..."
510,Wonogiri,None,22746128B32217084331074,IDN,ADM2,33,3312,JAWA TENGAH,WONOGIRI,JAVA & BALI,"POLYGON ((110.82982 -8.20142, 110.82924 -8.194..."
511,Wonosobo,None,22746128B92173297199140,IDN,ADM2,33,3307,JAWA TENGAH,WONOSOBO,JAVA & BALI,"POLYGON ((109.75216 -7.46188, 109.75625 -7.449..."
512,Yahukimo,None,22746128B87177062694316,IDN,ADM2,97,9707,PAPUA PEGUNUNGAN,YAHUKIMO,PAPUA,"POLYGON ((139.64443 -5.04107, 139.5114 -5.0063..."


In [41]:
admin_shp.sindex

In [42]:
rook = 0
queen = 1

In [43]:
admin_shp.geometry

0      POLYGON ((96.49109 4.62277, 96.49445 4.60326, ...
1      MULTIPOLYGON (((96.80559 3.71758, 96.80286 3.7...
2      MULTIPOLYGON (((95.20544 5.28072, 95.20757 5.2...
3      MULTIPOLYGON (((95.58431 4.61495, 95.58182 4.6...
4      MULTIPOLYGON (((97.59461 2.80777, 97.59269 2.8...
                             ...                        
509    POLYGON ((104.50089 -4.95499, 104.50019 -4.949...
510    POLYGON ((110.82982 -8.20142, 110.82924 -8.194...
511    POLYGON ((109.75216 -7.46188, 109.75625 -7.449...
512    POLYGON ((139.64443 -5.04107, 139.5114 -5.0063...
513    POLYGON ((139.46135 -3.45843, 139.47018 -3.456...
Name: geometry, Length: 514, dtype: geometry

In [44]:
# Initialize empty adjacency matrix
n = len(admin_shp)
adj = np.zeros((n, n), dtype=int)

for i, geom_i in enumerate(admin_shp.geometry):
    # Only check likely neighbors using spatial index
    candidates = list(admin_shp.sindex.intersection(geom_i.bounds))
    
    for j in candidates:
        if i == j:
            continue
        
        geom_j = admin_shp.geometry[j]

        # Rook adjacency: check shared **boundary segment**
        if geom_i.touches(geom_j):
            if rook:
                # touches returns True even for point-touching, so refine:
                inter = geom_i.intersection(geom_j)
                if inter.geom_type in ["LineString", "MultiLineString"]:
                    adj[i, j] = 1
                    adj[j, i] = 1
            elif queen:
                adj[i, j] = 1
                adj[j, i] = 1
            else:
                raise ValueError("Specify either rook or queen adjacency.")

In [46]:
# Convert to DataFrame with labeled index & columns
labels = admin_shp['admin2'].tolist()
adj_df = pd.DataFrame(adj, index=labels, columns=labels)

# Optional: ensure matrix is integer type (0/1)
adj_df = adj_df.astype(int)

adj_df.head()

,ACEH BARAT,ACEH BARAT DAYA,ACEH BESAR,ACEH JAYA,ACEH SELATAN,ACEH SINGKIL,ACEH TAMIANG,ACEH TENGAH,ACEH TENGGARA,ACEH TIMUR,...,TULANGBAWANG,TULUNGAGUNG,WAJO,WAKATOBI,WAROPEN,WAY KANAN,WONOGIRI,WONOSOBO,YAHUKIMO,YALIMO
ACEH BARAT,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ACEH BARAT DAYA,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACEH BESAR,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACEH JAYA,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACEH SELATAN,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
adj_df.to_csv(f'base_info/admin2_adjacency.csv')